In [ ]:
import os

import netCDF4 as nc

import math
import numpy as np
import numpy.ma as ma

import matplotlib.pyplot as plt
from matplotlib import cm

import cartopy.crs as ccrs
import cartopy.feature as cf
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

from myfunctions import *

from scipy import interpolate
from scipy.interpolate import griddata

In [ ]:
######### EXTRACTION DES DONNEES DE COUVERTURE NUAGEUSE Cloud_Sat ##########

# Where to save images
rep0 = './quicklooks/'
# If the directory does not exist, we create it
if not(os.path.exists(rep0)):
    os.makedirs(rep0)

# Projection for plotting
proj = ccrs.LambertConformal(central_latitude = 37,
                             central_longitude = 10,
                             standard_parallels = (37, 37)
                             )

var1 = 'cll'
var2 = 'clm'
var3 = 'clh'
var4 = 'clt'

data4d = np.zeros((120,101,153,4))
i=0
for annee in range (2007,2017):
    for mois in range (1,13):
        if mois<10:
            fin = 'CF_LMH_CloudSat_CALIPSO_'+str(annee)+"0"+str(mois)+'_AAD50.nc'
        else :
            fin = 'CF_LMH_CloudSat_CALIPSO_'+str(annee)+str(mois)+'_AAD50.nc'
        d = nc.Dataset(fin)
        
        data1 = d[var1][:,:,:]
        data2 = d[var2][:,:,:]
        data3 = d[var3][:,:,:]
        data4 = d[var4][:,:,:]
        data4d[i,:,:,0] = data1[0,:,:]
        data4d[i,:,:,1] = data2[0,:,:]
        data4d[i,:,:,2] = data3[0,:,:]
        data4d[i,:,:,3] = data4[0,:,:]
        i=i+1

lat = d['lat'][:,:]
lon = d['lon'][:,:]

d.close()


##### -- to nan #####

data_filtre = np.zeros((120,101,153,4))
data_filtre[:] = np.nan

for k in range(120):
    for i in range(101):
        for j in range(153):
            for l in range(4):
                if data4d[k,i,j,l]>=0 and data4d[k,i,j,l]<=1:
                    data_filtre[k,i,j,l] = data4d[k,i,j,l]
                else:
                    data_filtre[k,i,j,l] = np.nan 

##### RE INTERPOLATION SUR LA GRILLE ALADIN ######

# Suppression des valeurs aberrantes et des trous, méthode linéaire

#data_inter = (time,lat,lon,high)    le 4eme est la couverture totale
data_inter = np.zeros((120,101,153,4))
data_inter[:] = np.nan


for i in range(48):
    for j in range(4):
        xx = lon[np.isfinite(data_filtre[i,:,:,j])].ravel()
        yy = lat[np.isfinite(data_filtre[i,:,:,j])].ravel()
        dd = data_filtre[i,:,:,j][np.isfinite(data_filtre[i,:,:,j])].ravel()
        data_inter[i,:,:,j] = griddata((yy, xx),dd,(lat, lon), method='linear')
        
for i in range(49,52):
    for j in range(4):
        xx = lon[np.isfinite(data_filtre[i,:,:,j])].ravel()
        yy = lat[np.isfinite(data_filtre[i,:,:,j])].ravel()
        dd = data_filtre[i,:,:,j][np.isfinite(data_filtre[i,:,:,j])].ravel()
        data_inter[i,:,:,j] = griddata((yy, xx),dd,(lat, lon), method='linear')
        
for i in range(65,120):
    for j in range(4):
        xx = lon[np.isfinite(data_filtre[i,:,:,j])].ravel()
        yy = lat[np.isfinite(data_filtre[i,:,:,j])].ravel()
        dd = data_filtre[i,:,:,j][np.isfinite(data_filtre[i,:,:,j])].ravel()
        data_inter[i,:,:,j] = griddata((yy, xx),dd,(lat, lon), method='linear')
        
#data_inter à exporter sous format nc


In [ ]:
######### EXTRACTION DES DONNEES DE COUVERTURE NUAGEUSE ISCCP ##########

# Where to save images
rep0 = './quicklooks/'
# If the directory does not exist, we create it
if not(os.path.exists(rep0)):
    os.makedirs(rep0)

# Projection for plotting
proj = ccrs.LambertConformal(central_latitude = 37,
                             central_longitude = 10,
                             standard_parallels = (37, 37)
                            )

var1 = 'cltisccp'
var2 = 'albisccp'
var3 = 'pctisccp'
var = 'clt'

#data3d = (time,lat,lon,var)
data3d = np.zeros((300,101,153,3))
data3d[:] = np.nan

fin = 'cltisccp_198307-200806_regrid_ALADIN_bilinear.nc'
fin2 = 'albisccp_198307-200806_regrid_ALADIN_bilinear.nc'
fin3 = 'pctisccp_198307-200806_regrid_ALADIN_bilinear.nc'

d = nc.Dataset(fin)
d2 = nc.Dataset(fin2)
d3 = nc.Dataset(fin3)

data = d[var1][:,:,:]
data2 = d2[var2][:,:,:]
data3 = d3[var3][:,:,:]

data3d[:,:,:,0] = data[:,:,:]
data3d[:,:,:,1] = data2[:,:,:]
data3d[:,:,:,2] = data3[:,:,:]

lat = d['latitude'][:,:]
lon = d['longitude'][:,:]

d.close()
d2.close()
d3.close()


### -- to nan ###

#data_filtre = (time,lat,lon,var)
data_filtre = np.zeros((300,101,153,3))
data_filtre[:] = np.nan

for k in range(300):
    for i in range(101):
        for j in range(153):
            for m in range(3):
                if data3d[k,i,j,m]>=0 and data3d[k,i,j,m]<=100000:
                    data_filtre[k,i,j,m] = data3d[k,i,j,m]
                else:
                    data_filtre[k,i,j,m] = np.nan 

### on supprime le bug du mois de juin (abscence de donnees au nord) ###              
                
for a in range(25):
    for i in range(101):
        for j in range(153):
            if Lat[i,j]>53:
                if data_filtre[12*a+5,i,j,0]<60:
                    data_filtre[12*a+5,i,j,0] = np.nan
                if data_filtre[12*a+5,i,j,2]<40000:
                    data_filtre[12*a+5,i,j,2] = np.nan
                    
#data_filtre à convertir en fichier nc                

In [ ]:
######### EXTRACTION DES DONNEES DE COUVERTURE NUAGEUSE MODIS ##########

# Where to save images
rep0 = './quicklooks/'
# If the directory does not exist, we create it
if not(os.path.exists(rep0)):
    os.makedirs(rep0)

# Projection for plotting
proj = ccrs.LambertConformal(central_latitude = 37,
                             central_longitude = 10,
                             standard_parallels = (37, 37)
                             )

var1 = 'Cloud_Fraction_Mask_Low_Mean'
var2 = 'Cloud_Fraction_Mask_Mid_Mean'
var3 = 'Cloud_Fraction_Mask_High_Mean'
var4 = 'Cloud_Fraction_Mask_Total_Mean'
var5 = 'Cloud_Optical_Thickness_Total_Mean'
var6 = 'Cloud_Top_Pressure_Total_Mean'

data4d = np.zeros((96,180,360,6))
i=0
for annee in range (2007,2015):
    for mois in range (1,13):
        if mois<10:
            fin = 'MODIS/MCD08_M3_NC.'+str(annee)+".0"+str(mois)+'.C051.V02.nc'
        else :
            fin = 'MODIS/MCD08_M3_NC.'+str(annee)+"."+str(mois)+'.C051.V02.nc'
        d = nc.Dataset(fin)
        
        data1 = d[var1][:,:]
        data2 = d[var2][:,:]
        data3 = d[var3][:,:]
        data4 = d[var4][:,:]
        data5 = d[var5][:,:]
        data6 = d[var6][:,:]
        data4d[i,:,:,0] = data1[:,:]
        data4d[i,:,:,1] = data2[:,:]
        data4d[i,:,:,2] = data3[:,:]
        data4d[i,:,:,3] = data4[:,:]
        data4d[i,:,:,4] = data5[:,:]
        data4d[i,:,:,5] = data6[:,:]
        i=i+1

lat = d['lat'][:]
lon = d['lon'][:]
Lon,Lat = np.meshgrid(lon,lat)

d.close()

print(data1.shape)
print(Lon)
##### -- to nan #####

data_filtre = np.zeros((96,180,360,6))
data_filtre[:] = np.nan

for k in range(96):
    for i in range(101):
        for j in range(153):
            for l in range(4):
                if data4d[k,i,j,l]>=0 and data4d[k,i,j,l]<=1:
                    data_filtre[k,i,j,l] = data4d[k,i,j,l]
                else:
                    data_filtre[k,i,j,l] = np.nan
data_filtre[:,:,:,4] = data4d[:,:,:,4]
data_filtre[:,:,:,5] = data4d[:,:,:,5]

##### RE INTERPOLATION SUR LA GRILLE ALADIN ######

# grille ALADIN

fin = 'rsut_CERES_EBAF_Ed4-1_200003-201901_regrid_ALADIN_bilinear.nc'
d = nc.Dataset(fin)
lat_A = d['lat'][:]
lon_A = d['lon'][:]
d.close()

# Suppression des valeurs aberrantes et des trous, méthode linéaire

#data = (time,lat,lon,variable)
data_inter = np.zeros((96,101,153,6))
data_inter[:] = np.nan

for i in range(96):
    for j in range(6):
        xx = Lon[np.isfinite(data4d[i,:,:,j])].ravel()
        yy = Lat[np.isfinite(data4d[i,:,:,j])].ravel()
        dd = data4d[i,:,:,j][np.isfinite(data4d[i,:,:,j])].ravel()
        data_inter[i,:,:,j] = griddata((yy, xx),dd,(lat_A, lon_A), method='linear')
        
# data_inter à convertir en nc avec lat_A et lon_A comme grille

In [ ]:
######### EXTRACTION DES DONNEES DE RAYONNEMENT EN SURFACE CERES ##########


# Where to save images
rep0 = './quicklooks/'
# If the directory does not exist, we create it
if not(os.path.exists(rep0)):
    os.makedirs(rep0)

# Projection for plotting
proj = ccrs.LambertConformal(central_latitude = 37,
                             central_longitude = 10,
                             standard_parallels = (37, 37)
                             )
var0= 'rsut'
var1 = 'rsus'
var2 = 'rsuscs'
var3 = 'rlus'
var4 = 'rluscs'
var5 = 'rsds'
var6 = 'rsdscs_c'
var7 = 'rlds'
var8 = 'rldscs'

# file to open
#fin = '{0}_MED-44_ECMWF-ERAINT_evaluation_r1i1p1f2_CNRM-ALADIN64_v1_mon_200701-200701.nc'.format(var)

fin1 = '{0}_CERES_EBAF_Ed4-1_200003-201901_regrid_ALADIN_bilinear.nc'.format(var1)   
fin2 = '{0}_CERES_EBAF_Ed4-1_200003-201901_regrid_ALADIN_bilinear.nc'.format(var2)
fin3 = '{0}_CERES_EBAF_Ed4-1_200003-201901_regrid_ALADIN_bilinear.nc'.format(var3)
fin4 = '{0}_CERES_EBAF_Ed4-1_200003-201901_regrid_ALADIN_bilinear.nc'.format(var3)
fin5 = '{0}_CERES_EBAF_Ed4-1_200003-201901_regrid_ALADIN_bilinear.nc'.format(var5)   
fin6 = '{0}_CERES_EBAF_Ed4-1_200003-201901_regrid_ALADIN_bilinear.nc'.format(var6)
fin7 = '{0}_CERES_EBAF_Ed4-1_200003-201901_regrid_ALADIN_bilinear.nc'.format(var7)
fin8 = '{0}_CERES_EBAF_Ed4-1_200003-201901_regrid_ALADIN_bilinear.nc'.format(var8)

# Open file as a dataset
a = nc.Dataset(fin1)
b = nc.Dataset(fin2)
c = nc.Dataset(fin3)
d = nc.Dataset(fin4)
e = nc.Dataset(fin5)
f = nc.Dataset(fin6)
g = nc.Dataset(fin7)
h = nc.Dataset(fin8)

# Read data, latitude, longitude, time
data1 = a[var0][:,:,:]
data2 = b[var0][:,:,:]
data3 = c[var0][:,:,:]
data4 = d[var0][:,:,:]
data5 = e[var0][:,:,:]
data6 = f[var0][:,:,:]
data7 = g[var0][:,:,:]
data8 = h[var0][:,:,:]


lat = d['lat'][:,:]
lon = d['lon'][:,:]
time = d['time'][:]


print(var1,ma.min(data1),ma.max(data1))
print(var2,ma.min(data2),ma.max(data2))
print(var3,ma.min(data3),ma.max(data3))
print(var4,ma.min(data4),ma.max(data4))
print(var5,ma.min(data5),ma.max(data5))
print(var6,ma.min(data6),ma.max(data6))
print(var7,ma.min(data7),ma.max(data7))
print(var8,ma.min(data8),ma.max(data8))

a.close()
b.close()
c.close()
d.close()
e.close()
f.close()
g.close()
h.close()

######### EXTRACTION DES DONNEES DE RAYONNEMENT TOA CERES ##########


# Where to save images
rep0 = './quicklooks/'
# If the directory does not exist, we create it
if not(os.path.exists(rep0)):
    os.makedirs(rep0)

# Projection for plotting
proj = ccrs.LambertConformal(central_latitude = 37,
                             central_longitude = 10,
                             standard_parallels = (37, 37)
                             )
var0= 'rsut'
var9 = 'rsut'
var10 = 'rsutcs'
var11 = 'rlut'
var12 = 'rlutcs'

# file to open
#fin = '{0}_MED-44_ECMWF-ERAINT_evaluation_r1i1p1f2_CNRM-ALADIN64_v1_mon_200701-200701.nc'.format(var)

fin9 = '{0}_CERES_EBAF_Ed4-1_200003-201901_regrid_ALADIN_bilinear.nc'.format(var9)   
fin10 = '{0}_CERES_EBAF_Ed4-1_200003-201901_regrid_ALADIN_bilinear.nc'.format(var10)
fin11 = '{0}_CERES_EBAF_Ed4-1_200003-201901_regrid_ALADIN_bilinear.nc'.format(var11)
fin12 = '{0}_CERES_EBAF_Ed4-1_200003-201901_regrid_ALADIN_bilinear.nc'.format(var12)


# Open file as a dataset
i = nc.Dataset(fin9)
j = nc.Dataset(fin10)
k = nc.Dataset(fin11)
l = nc.Dataset(fin12)


# Read data, latitude, longitude, time
data9 = i[var0][:,:,:]
data10 = j[var0][:,:,:]
data11 = k[var0][:,:,:]
data12 = l[var0][:,:,:]




print(var9,ma.min(data9),ma.max(data9))
print(var10,ma.min(data10),ma.max(data10))
print(var11,ma.min(data11),ma.max(data11))
print(var12,ma.min(data12),ma.max(data12))


i.close()
j.close()
k.close()
l.close()

print(data1[82:202].shape)

#Data = (variable,time,lat,lon)
Data = np.zeros((13,120,101,153))
Data[:] = np.nan

data = [data1[82:202],data2[82:202],data3[82:202],data4[82:202],\
        data5[82:202],data6[82:202],data7[82:202],data8[82:202],\
        data9[82:202],data10[82:202],data11[82:202],data12[82:202],data_rsdt]

for i in range(13):
    Data[i,:,:,:] = data[i]

# Data à convertir en nc
